#### Import the libraries

In [35]:
import os
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive/')
# método em Python usado para alterar o diretório de trabalho atual para o caminho especificado.
path = "/content/drive/MyDrive/Artigos_IC"
os.chdir(path)
import shutil
shutil.copy('/content/drive/MyDrive/Artigos_IC/out.csv', '/content/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/out.csv'

In [36]:
!pip install pycaret

In [37]:
!apt-get update
!apt-get install -y wget unzip
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

!wget -q https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/


Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,220 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 387 kB in 2s (196 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main

In [38]:
!pip install selenium

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")  # Executar em modo headless
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

chrome_path = '/usr/bin/google-chrome'  # Caminho do Chrome no Colab
webdriver_service = Service(executable_path='/usr/local/bin/chromedriver')  # Caminho do ChromeDriver no Colab

#driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)


In [40]:
!pip install optuna-integration

In [41]:
!pip install dataframe_image

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dataframe_image as dfi
from pycaret.classification import *
import ast

In [43]:
df = pd.read_csv('/content/out.csv', index_col=0)
#df_type = df['Type']
df.shape

(17833, 191)

#### Remove duplicates

In [44]:
duplicate_rows = df[df.duplicated()]
len(duplicate_rows)

136

In [45]:
df.columns.values
df.columns=df.columns.str.replace(' ','_')

In [46]:
df = df.drop('start_line',axis=1)
df = df.drop('end_line',axis=1)
df = df.drop('commit_hash',axis=1)
df = df.drop('agreement_experts_has_smell',axis=1)
df = df.drop('agreement_has_smell', axis=1)
df = df.drop('id',axis=1)
df = df.drop('smell',axis=1)
df = df.drop('severity',axis=1)
df = df.drop('reviewer_id',axis=1)
df = df.drop('sample_id',axis=1)
df = df.drop('review_timestamp',axis=1)
df = df.drop('code_name',axis=1)
df = df.drop('repository',axis=1)
df = df.drop('path',axis=1)
df = df.drop('link',axis=1)
df = df.drop('is_from_industry_relevant_project',axis=1)
df = df.drop('organic_filerelativepath',axis=1)
df = df.drop('organic_startline', axis=1)
df = df.drop('organic_endline', axis=1)
df = df.drop('pmd_package', axis=1)
df = df.drop('pmd_file', axis=1)
df = df.drop('pmd_line', axis=1)
df = df.drop('pmd_description',axis=1)
df = df.drop('pmd_rule_set',axis=1)
df = df.drop('implementation_project_name',axis=1)
df = df.drop('implementation_code_name',axis=1)
df = df.drop('implementation_type_name',axis=1)
df = df.drop('implementation_method_name',axis=1)
df = df.drop('implementation_method_start_line_no',axis=1)
df = df.drop('implementation_cause_of_the_smell',axis=1)
df = df.drop('designite_project_name',axis=1)
df = df.drop('designite_implementation_smell',axis=1)
df = df.drop('designite_design_smell',axis=1)
df = df.drop('designite_package_name',axis=1)
df = df.drop('designite_type_name',axis=1)
df = df.drop('ck_class_file',axis=1)
df = df.drop('ck_method_file',axis=1)
df = df.drop('ck_method_class',axis=1)
df = df.drop('ck_method',axis=1)
df = df.drop('ck_class',axis=1)
df = df.drop('ck_class_type',axis=1)
df = df.drop('ck_method_constructor',axis=1)
df = df.drop('ck_method_line',axis=1)
df = df.drop('designite_cause_of_the_smell',axis=1)
df = df.drop('system',axis=1)
df = df.drop('system_name',axis=1)

In [47]:
filtered_columns = [cols for cols in df.columns if cols.startswith('ck')]
# Remove filtered columns
def parse_list(string):
    try:
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        return string

def treat_column(val):
    if isinstance(val, list):
        return sum(val)
    else:
        return val

for col in filtered_columns:
    df[col] = df[col].apply(parse_list)
    df[col] = df[col].apply(treat_column)

## Drop Code Smell - LongMethod

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

In [ ]:
#consider only the column 'longmethod_label'

df_longMethod = df.drop(columns=['featureenvy_label','dataclass_label','blob_label'])
df_longMethod.columns


Index(['longmethod_label', 'agreement_smell_type', 'type',
       'organic_weighofclass', 'organic_numberofaccessormethods',
       'organic_classeffectivelinesofcode', 'organic_lcom3',
       'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount',
       ...
       'smell_organic_dataclass', 'smell_organic_blob', 'smell_pmd_longmethod',
       'smell_pmd_featureenvy', 'smell_pmd_dataclass', 'smell_pmd_blob',
       'smell_designite_longmethod', 'smell_designite_featureenvy',
       'smell_designite_dataclass', 'smell_designite_blob'],
      dtype='object', length=142)

In [ ]:
print(df['ck_method_cbomodified'])

system_id
0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
11351    3.0
11352    3.0
11353    3.0
11354    3.0
11355    3.0
Name: ck_method_cbomodified, Length: 17833, dtype: float64


#### Check the correlation and remove the high-correlated features from the dataset

In [ ]:
corr = df_longMethod.corr(numeric_only = True)
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_longMethod.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_longMethod.columns[columns]
high_corr = set(df_longMethod.columns) - set(selected_columns)
df_longMethod = df_longMethod[selected_columns]
df_longMethod.columns

Index(['longmethod_label', 'agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethodsq

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

In [ ]:
results = df_longMethod.longmethod_label.values
df_longMethod = df_longMethod.drop('longmethod_label', axis=1)
df_longMethod['longmethod_label'] = results
df_longMethod.columns

Index(['agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethodsqty',
       'ck_clas

#### Setup the model creation with pycaret

In [ ]:
s = setup(data=df_longMethod, target='longmethod_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Info] Number of positive: 26109, number of negative: 26109
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024626 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7427
[LightGBM] [Info] Number of data points in the train set: 52218, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,3018
1,Target,longmethod_label
2,Target type,Binary
3,Original data shape,"(40951, 86)"
4,Transformed data shape,"(64504, 18)"
5,Transformed train set shape,"(52218, 18)"
6,Transformed test set shape,"(12286, 18)"
7,Numeric features,85
8,Rows with missing values,100.0%
9,Preprocess,True


In [ ]:
df_longMethod = get_config('X')
df_longMethod

,agreement_smell_type,organic_weighofclass,organic_numberofaccessormethods,organic_classeffectivelinesofcode,organic_lcom3,organic_tightclasscohesion,organic_isabstract,organic_publicfieldcount,organic_weightedmethodcount,organic_changingmethods,...,smell_organic_dataclass,smell_organic_blob,smell_pmd_longmethod,smell_pmd_featureenvy,smell_pmd_dataclass,smell_pmd_blob,smell_designite_longmethod,smell_designite_featureenvy,smell_designite_dataclass,smell_designite_blob
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,0,0,0,0,0,0,0,0,0,0
40947,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [ ]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8377,0.9416,0.9511,0.3500,0.5115,0.4382,0.5174,8.7470
et,Extra Trees Classifier,0.8358,0.9404,0.9468,0.3465,0.5072,0.4332,0.5124,7.5520
xgboost,Extreme Gradient Boosting,0.8339,0.9413,0.9487,0.3441,0.5049,0.4303,0.5106,3.1830
dt,Decision Tree Classifier,0.8328,0.9373,0.9421,0.3417,0.5014,0.4263,0.5059,2.7240
lightgbm,Light Gradient Boosting Machine,0.8287,0.9407,0.9472,0.3367,0.4967,0.4204,0.5025,6.0520
knn,K Neighbors Classifier,0.9078,0.8814,0.4092,0.4819,0.4417,0.3919,0.3939,4.6980
gbc,Gradient Boosting Classifier,0.7598,0.9028,0.8979,0.2574,0.4001,0.3036,0.3966,10.6860
ada,Ada Boost Classifier,0.7322,0.8657,0.8427,0.2291,0.3601,0.2557,0.3427,4.7200
lda,Linear Discriminant Analysis,0.6586,0.7275,0.6432,0.1585,0.2531,0.1284,0.1810,2.5600
ridge,Ridge Classifier,0.6509,0.7244,0.6584,0.1557,0.2518,0.1257,0.1815,2.6010


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongMethod_FS2.latex',index=False)
df3.to_csv('LongMethod_FS2.csv',index=False)
dfi.export(df2, 'LongMethod_FS2.png')

#### Tune the models

In [ ]:
tuned_top5 = []
results=[]

In [ ]:
for model in top5_models:
    if model.__class__.__name__ == 'GaussianNB':
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=False  # Desativar early_stopping para Naive Bayes
        )
    else:
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=True
        )
    tuned_top5.append(tuned_model)
    best_model_results = pull()
    metrics = best_model_results.iloc[:, :-1].T.to_dict('list')
    params = tuned_model.get_params()

    result = {
        "Model": model.__class__.__name__,
        "Parameters": params
    }
    result.update(metrics)

    results.append(result)

In [ ]:
df3 = pd.DataFrame(results)
df3.to_latex('LongMethod_FS2_Top_5.latex',index=False)
dfi.export(df2, 'LongMethod_FS2_Top_5.png')

#### Calculing top 5 most relevant features

##### Random Forest (RF)

In [ ]:
best_model = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8322,0.9405,0.9727,0.3444,0.5087,0.4340,0.5195
1,0.8280,0.9459,0.9492,0.3361,0.4964,0.4199,0.5026
2,0.8340,0.9369,0.9492,0.3442,0.5052,0.4306,0.5109
3,0.8364,0.9457,0.9688,0.3498,0.5140,0.4406,0.5236
4,0.8357,0.9375,0.9766,0.3497,0.5149,0.4415,0.5263
5,0.8133,0.9262,0.9255,0.3138,0.4687,0.3873,0.4710
6,0.8276,0.9267,0.9255,0.3319,0.4886,0.4115,0.4902
7,0.8360,0.9433,0.9451,0.3458,0.5063,0.4323,0.5113
8,0.8423,0.9422,0.9569,0.3562,0.5191,0.4475,0.5260


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongMethod_RF_FS2.latex',index=False)
df3.to_csv('LongMethod_RF_FS2.csv',index=False)
dfi.export(df2, 'LongMethod_RF_FS2.png')

##### Extra Trees Classifier (ET)

In [ ]:
best_model_et= create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8333,0.9396,0.9531,0.3437,0.5052,0.4304,0.5118
1,0.8249,0.9332,0.9570,0.3329,0.4940,0.4167,0.5020
2,0.8430,0.9470,0.9492,0.3574,0.5192,0.4476,0.5242
3,0.8375,0.9443,0.9688,0.3513,0.5156,0.4425,0.5251
4,0.8172,0.9164,0.9375,0.3209,0.4781,0.3980,0.4825
5,0.8109,0.9280,0.9490,0.3139,0.4717,0.3902,0.4792
6,0.8238,0.9314,0.9216,0.3264,0.4821,0.4037,0.4830
7,0.8343,0.9427,0.9569,0.3446,0.5067,0.4325,0.5143
8,0.8468,0.9472,0.9569,0.3631,0.5264,0.4563,0.5329


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df3 = pd.DataFrame(results)
df3.to_latex('LongMethod_FS2_Top_5.latex',index=False)
dfi.export(df2, 'LongMethod_FS2_Top_5.png')

##### Decision Tree (DT)

In [ ]:
best_model_dt= create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8253,0.9306,0.9648,0.3342,0.4965,0.4195,0.5062
1,0.8301,0.9426,0.9492,0.3389,0.4995,0.4236,0.5055
2,0.8385,0.9459,0.9453,0.3502,0.5111,0.4378,0.5156
3,0.8364,0.9457,0.9688,0.3498,0.5140,0.4406,0.5236
4,0.8270,0.9351,0.9688,0.3370,0.5000,0.4236,0.5104
5,0.8088,0.9241,0.9294,0.3090,0.4638,0.3812,0.4672
6,0.8269,0.9267,0.9255,0.3310,0.4876,0.4103,0.4892
7,0.8364,0.9411,0.9608,0.3480,0.5109,0.4375,0.5192
8,0.8398,0.9391,0.9529,0.3522,0.5143,0.4418,0.5206


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongMethod_DT_FS2.latex',index=False)
df3.to_csv('LongMethod_DT_FS2.csv',index=False)
dfi.export(df2, 'LongMethod_DT_FS2.png')

##### Extreme Gradient Boosting	(xgboost)

In [ ]:
best_model_gbc = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8225,0.9322,0.9648,0.3307,0.4925,0.4147,0.5024
1,0.8343,0.9449,0.9492,0.3447,0.5057,0.4312,0.5114
2,0.8403,0.9483,0.9531,0.3536,0.5159,0.4433,0.5219
3,0.8371,0.9490,0.9609,0.3499,0.5130,0.4397,0.5209
4,0.8270,0.9352,0.9727,0.3374,0.5010,0.4247,0.5123
5,0.8057,0.9204,0.9451,0.3074,0.4639,0.3808,0.4708
6,0.8266,0.9326,0.9333,0.3315,0.4892,0.4120,0.4925
7,0.8297,0.9396,0.9373,0.3361,0.4948,0.4187,0.4987
8,0.8388,0.9408,0.9608,0.3515,0.5147,0.4420,0.5227


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongMethod_xgboost_FS2.latex',index=False)
df3.to_csv('LongMethod_xgboost_FS2.csv',index=False)
dfi.export(df2, 'LongMethodxgboost_FS2.png')

##### Light Gradient Boosting Machine (lightgbm)

In [ ]:
best_model_ada = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8242,0.9394,0.9648,0.3329,0.4950,0.4177,0.5048
1,0.8232,0.9430,0.9531,0.3302,0.4905,0.4125,0.4978
2,0.8368,0.9470,0.9570,0.3490,0.5115,0.4379,0.5186
3,0.8270,0.9406,0.9453,0.3343,0.4939,0.4170,0.4993
4,0.8197,0.9365,0.9688,0.3276,0.4896,0.4110,0.5006
5,0.7938,0.9176,0.9608,0.2966,0.4533,0.3672,0.4641
6,0.8179,0.9335,0.9373,0.3208,0.4780,0.3982,0.4826
7,0.8287,0.9406,0.9412,0.3352,0.4943,0.4180,0.4991
8,0.8381,0.9457,0.9608,0.3505,0.5136,0.4407,0.5217


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongMethod_lightgbm_FS2.latex',index=False)
df3.to_csv('LongMethod_lightgbm_FS2.csv',index=False)
dfi.export(df2, 'LongMethod_lightgbm_FS2.png')

In [ ]:
plot_model(best_model_ada, plot='feature',save=True)
os.rename('Feature Importance.png','LongMethod_lightgbm.png')

In [ ]:
plot_model(best_model_gbc, plot='feature',save=True)
os.rename('Feature Importance.png','LongMethod_gbc.png')

In [ ]:
plot_model(best_model_dt, plot='feature',save=True)
os.rename('Feature Importance.png','LongMethod_dt.png')

In [ ]:
plot_model(best_model, plot='feature',save=True)
os.rename('Feature Importance.png','LongMethod_rf.png')

In [ ]:
plot_model(best_model_et, plot='feature',save=True)
os.rename('Feature Importance.png','LongMethod_et.png')

#### Ensemble the top-5 models

In [ ]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8413,0.9395,0.9219,0.3517,0.5092,0.4363,0.5087
1,0.8280,0.9384,0.9648,0.3379,0.5005,0.4244,0.5100
2,0.8458,0.9509,0.9609,0.3628,0.5268,0.4563,0.5338
3,0.8378,0.9427,0.9297,0.3474,0.5058,0.4320,0.5072
4,0.8500,0.9475,0.9688,0.3701,0.5356,0.4667,0.5439
5,0.8391,0.9453,0.9255,0.3481,0.5059,0.4325,0.5066
6,0.8290,0.9358,0.9725,0.3393,0.5030,0.4275,0.5144
7,0.8371,0.9334,0.9373,0.3464,0.5058,0.4320,0.5091
8,0.8255,0.9344,0.9608,0.3333,0.4949,0.4181,0.5041


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8305,0.9376,0.9516,0.3394,0.5004,0.4247,0.5069


#### Plot the ensemble model

In [ ]:
result = pull()

In [ ]:
result.to_csv('models_longmethod2.csv')

In [ ]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_longmethod_2')

IndexError: Length of values (74598) does not match length of index (40951). This usually happens when transformations that drop rows aren't applied on all the columns.

#### Finalize and save the ensemble model

#### Interpret the ensemble model

In [ ]:
!pip install shap
interpret_model(top5_models[0], save=True)

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

## Drop Code Smell - FeatureEnvy

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the  Feature Envy smell. To train other models, remove the other code smells.

In [ ]:
#consider only the column 'featureenvy_label'
df.columns.values
df_featureenvy = df.drop(columns=['longmethod_label','dataclass_label','blob_label'])
df_featureenvy.columns

Index(['featureenvy_label', 'agreement_smell_type', 'type',
       'organic_weighofclass', 'organic_numberofaccessormethods',
       'organic_classeffectivelinesofcode', 'organic_lcom3',
       'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount',
       ...
       'smell_organic_dataclass', 'smell_organic_blob', 'smell_pmd_longmethod',
       'smell_pmd_featureenvy', 'smell_pmd_dataclass', 'smell_pmd_blob',
       'smell_designite_longmethod', 'smell_designite_featureenvy',
       'smell_designite_dataclass', 'smell_designite_blob'],
      dtype='object', length=142)

#### Check the correlation and remove the high-correlated features from the dataset

In [ ]:
corr = df_featureenvy.corr(numeric_only = True)
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_featureenvy.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_featureenvy.columns[columns]
high_corr = set(df_featureenvy.columns) - set(selected_columns)
df_featureenvy = df_featureenvy[selected_columns]
df_featureenvy.columns

Index(['featureenvy_label', 'agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethods

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

In [ ]:
results = df_featureenvy.featureenvy_label.values
df_featureenvy = df_featureenvy.drop('featureenvy_label', axis=1)
df_featureenvy['featureenvy_label'] = results
df_featureenvy.columns

Index(['agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethodsqty',
       'ck_clas

#### Setup the model creation with pycaret

In [ ]:
s = setup(data=df_featureenvy, target='featureenvy_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Info] Number of positive: 26088, number of negative: 26088
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5042
[LightGBM] [Info] Number of data points in the train set: 52176, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,5226
1,Target,featureenvy_label
2,Target type,Binary
3,Original data shape,"(40951, 86)"
4,Transformed data shape,"(64462, 18)"
5,Transformed train set shape,"(52176, 18)"
6,Transformed test set shape,"(12286, 18)"
7,Numeric features,85
8,Rows with missing values,100.0%
9,Preprocess,True


In [ ]:
df_featureenvy = get_config('X')
df_featureenvy

,agreement_smell_type,organic_weighofclass,organic_numberofaccessormethods,organic_classeffectivelinesofcode,organic_lcom3,organic_tightclasscohesion,organic_isabstract,organic_publicfieldcount,organic_weightedmethodcount,organic_changingmethods,...,smell_organic_dataclass,smell_organic_blob,smell_pmd_longmethod,smell_pmd_featureenvy,smell_pmd_dataclass,smell_pmd_blob,smell_designite_longmethod,smell_designite_featureenvy,smell_designite_dataclass,smell_designite_blob
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [ ]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8112,0.9337,0.9825,0.3207,0.4835,0.4025,0.4974,8.6070
et,Extra Trees Classifier,0.8123,0.9335,0.9748,0.3211,0.4830,0.4022,0.4952,6.9460
dt,Decision Tree Classifier,0.8096,0.9301,0.9725,0.3178,0.4790,0.3973,0.4907,2.5360
xgboost,Extreme Gradient Boosting,0.8071,0.9346,0.9849,0.3163,0.4787,0.3966,0.4934,2.8740
lightgbm,Light Gradient Boosting Machine,0.7970,0.9338,0.9775,0.3043,0.4641,0.3789,0.4776,5.5680
knn,K Neighbors Classifier,0.8994,0.8678,0.3597,0.4309,0.3912,0.3369,0.3391,4.2690
gbc,Gradient Boosting Classifier,0.6993,0.8721,0.9228,0.2203,0.3556,0.2462,0.3542,9.6760
ada,Ada Boost Classifier,0.6715,0.8248,0.8362,0.1933,0.3140,0.1967,0.2884,4.2280
ridge,Ridge Classifier,0.6188,0.7200,0.6946,0.1500,0.2467,0.1160,0.1775,2.3550
lda,Linear Discriminant Analysis,0.6182,0.7198,0.6931,0.1495,0.2460,0.1152,0.1763,2.3610


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
df2=pull()
dfi.export(df2, 'FeatureEnvy_FS2.png')
df3 = pd.DataFrame(df2)
df3.to_csv('FeatureEnvy_FS2.csv',index=False)
df3.to_latex('FeatureEnvy_FS2.latex',index=False)
dfi.export(df2, 'FeatureEnvy_FS2.png')

#### Tune the models

In [ ]:
tuned_top5=[]
results=[]

In [ ]:
for model in top5_models:
    if model.__class__.__name__ == 'GaussianNB':
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=False  # Desativar early_stopping para Naive Bayes
        )
    else:
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=True
        )
    tuned_top5.append(tuned_model)
    best_model_results = pull()
    metrics = best_model_results.iloc[:, :-1].T.to_dict('list')
    params = tuned_model.get_params()

    result = {
        "Model": model.__class__.__name__,
        "Parameters": params
    }
    result.update(metrics)

    results.append(result)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8713,0.9292,0.9961,0.4112,0.5821,0.5211,0.5926
1,0.8755,0.9307,0.9729,0.4176,0.5844,0.5245,0.5897
2,0.8877,0.9378,0.9884,0.4443,0.6130,0.5581,0.6194
3,0.8786,0.9311,0.9690,0.4237,0.5896,0.5309,0.5937
4,0.8629,0.9220,0.9767,0.3944,0.5619,0.4974,0.5696
5,0.8824,0.9338,0.9883,0.4320,0.6012,0.5443,0.6087
6,0.8786,0.9253,0.9728,0.4230,0.5896,0.5310,0.5947
7,0.8636,0.9239,0.9883,0.3956,0.5651,0.5012,0.5754
8,0.8737,0.9310,0.9884,0.4153,0.5849,0.5246,0.5934


[I 2024-06-07 15:46:11,655] Searching the best hyperparameters using 28665 samples...
[I 2024-06-07 15:50:08,467] Finished hyperparameter search!


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8692,0.9311,0.9729,0.4055,0.5724,0.5102,0.5785
1,0.8706,0.9347,0.9302,0.4047,0.5640,0.5015,0.5616
2,0.8811,0.9400,0.9147,0.4252,0.5806,0.5218,0.5740
3,0.8675,0.9292,0.9109,0.3970,0.5529,0.4889,0.5472
4,0.8664,0.9256,0.9574,0.3990,0.5633,0.4997,0.5667
5,0.8807,0.9347,0.9339,0.4248,0.5839,0.5254,0.5812
6,0.8765,0.9315,0.9533,0.4174,0.5806,0.5208,0.5821
7,0.8643,0.9254,0.9533,0.3939,0.5575,0.4931,0.5606
8,0.8723,0.9323,0.9767,0.4118,0.5793,0.5183,0.5858


[I 2024-06-07 15:50:28,904] Searching the best hyperparameters using 28665 samples...
[I 2024-06-07 16:00:47,885] Finished hyperparameter search!


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8162,0.9139,0.9264,0.3199,0.4756,0.3946,0.4770
1,0.8169,0.9018,0.9186,0.3198,0.4745,0.3935,0.4742
2,0.8235,0.9176,0.9419,0.3311,0.4899,0.4116,0.4942
3,0.7991,0.9015,0.8915,0.2956,0.4440,0.3571,0.4385
4,0.8036,0.9029,0.9225,0.3047,0.4581,0.3734,0.4592
5,0.8238,0.9158,0.9300,0.3292,0.4863,0.4078,0.4883
6,0.8175,0.9002,0.9066,0.3183,0.4712,0.3902,0.4687
7,0.8001,0.9058,0.9339,0.3015,0.4558,0.3705,0.4598
8,0.8133,0.9144,0.9302,0.3170,0.4729,0.3912,0.4752


...

[I 2024-06-07 16:11:11,331] Searching the best hyperparameters using 28665 samples...
[I 2024-06-07 16:16:32,988] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8385,0.9083,0.8605,0.3421,0.4895,0.4141,0.4765
1,0.8420,0.9064,0.8411,0.3450,0.4893,0.4146,0.4724
2,0.8511,0.9222,0.9031,0.3669,0.5218,0.4517,0.5162
3,0.8165,0.8867,0.8023,0.3035,0.4404,0.3564,0.4169
4,0.8350,0.9054,0.8643,0.3374,0.4853,0.4088,0.4732
5,0.8444,0.9159,0.9455,0.3600,0.5215,0.4500,0.5252
6,0.8273,0.8863,0.8405,0.3224,0.4660,0.3865,0.4499
7,0.8346,0.9142,0.8872,0.3388,0.4903,0.4143,0.4830
8,0.8364,0.9124,0.8488,0.3374,0.4829,0.4064,0.4677


[I 2024-06-07 16:16:58,650] Searching the best hyperparameters using 28665 samples...
[I 2024-06-07 16:19:57,907] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
df3 = pd.DataFrame(results)
df3.to_latex('FeatureEnvy_FS2_Top_5.latex',index=False)
dfi.export(df2, 'FeatureEnvy_FS2_Top_5.png')

#### Calculing top 5 most relevant features

##### Random Forest (RF)

In [ ]:
best_model = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8092,0.9346,0.9922,0.3196,0.4835,0.4021,0.4996
1,0.8078,0.9251,0.9690,0.3153,0.4757,0.3934,0.4868
2,0.8130,0.9317,0.9884,0.3236,0.4876,0.4072,0.5026
3,0.8096,0.9320,0.9845,0.3191,0.4820,0.4005,0.4963
4,0.8176,0.9342,0.9767,0.3277,0.4907,0.4114,0.5029
5,0.8119,0.9280,0.9611,0.3183,0.4782,0.3970,0.4876
6,0.8025,0.9332,0.9767,0.3095,0.4700,0.3865,0.4833
7,0.8238,0.9422,0.9767,0.3347,0.4985,0.4212,0.5105
8,0.8116,0.9292,0.9806,0.3211,0.4837,0.4027,0.4971


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('FeatureEnvy_RF_FS2.latex',index=False)
dfi.export(df2, 'FeatureEnvy_RF_FS2.png')

##### Extra Trees Classifier (ET)

In [ ]:
best_model_et= create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8064,0.9322,0.9806,0.3151,0.4769,0.3944,0.4906
1,0.8075,0.9188,0.9767,0.3158,0.4773,0.3950,0.4900
2,0.8120,0.9301,0.9767,0.3210,0.4832,0.4023,0.4957
3,0.8096,0.9310,0.9845,0.3191,0.4820,0.4005,0.4963
4,0.8193,0.9352,0.9767,0.3298,0.4932,0.4144,0.5052
5,0.8077,0.9250,0.9650,0.3139,0.4737,0.3914,0.4842
6,0.8074,0.9343,0.9883,0.3163,0.4792,0.3974,0.4949
7,0.8186,0.9412,0.9805,0.3286,0.4922,0.4134,0.5054
8,0.8133,0.9294,0.9574,0.3204,0.4801,0.3990,0.4882


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('FeatureEnvy_ET_FS2.latex',index=False)
dfi.export(df2, 'FeatureEnvy_ET_FS2.png')

##### Decision Tree (DT)

In [ ]:
best_model_dt= create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8043,0.9315,1.0000,0.3150,0.4791,0.3965,0.4973
1,0.8019,0.9176,0.9767,0.3096,0.4701,0.3863,0.4832
2,0.8096,0.9282,0.9690,0.3173,0.4780,0.3961,0.4889
3,0.8106,0.9284,0.9806,0.3198,0.4824,0.4011,0.4958
4,0.8134,0.9356,0.9884,0.3240,0.4880,0.4078,0.5030
5,0.8039,0.9239,0.9728,0.3106,0.4708,0.3876,0.4832
6,0.7973,0.9289,0.9767,0.3039,0.4635,0.3785,0.4771
7,0.8214,0.9397,0.9767,0.3316,0.4951,0.4170,0.5072
8,0.8102,0.9274,0.9690,0.3181,0.4789,0.3972,0.4898


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('FeatureEnvy_DT_FS2.latex',index=False)
dfi.export(df2, 'FeatureEnvy_DT_FS2.png')

##### Extreme Gradient Boosting(xgboost)

In [ ]:
best_model_xgboost = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8033,0.9349,1.0000,0.3139,0.4778,0.3949,0.4960
1,0.8012,0.9214,0.9729,0.3084,0.4683,0.3841,0.4805
2,0.8089,0.9325,0.9806,0.3178,0.4801,0.3983,0.4936
3,0.8116,0.9312,0.9806,0.3211,0.4837,0.4028,0.4971
4,0.8113,0.9349,0.9845,0.3211,0.4843,0.4033,0.4985
5,0.8046,0.9243,0.9533,0.3090,0.4667,0.3831,0.4747
6,0.8018,0.9336,0.9805,0.3092,0.4701,0.3865,0.4844
7,0.8186,0.9417,0.9611,0.3263,0.4872,0.4079,0.4962
8,0.8032,0.9318,0.9767,0.3111,0.4719,0.3884,0.4849


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('FeatureEnvy_xgboost_FS2.latex',index=False)
dfi.export(df2, 'FeatureEnvy_xgboost_FS2.png')

##### Light Gradient Boosting Machine (lightgbm)

In [ ]:
best_model_lightgbm= create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7963,0.9331,0.9922,0.3055,0.4672,0.3821,0.4840
1,0.7928,0.9175,0.9690,0.2990,0.4570,0.3705,0.4687
2,0.7970,0.9295,0.9729,0.3039,0.4631,0.3778,0.4755
3,0.7932,0.9320,0.9845,0.3013,0.4614,0.3753,0.4766
4,0.8026,0.9343,0.9729,0.3099,0.4700,0.3863,0.4822
5,0.7966,0.9226,0.9494,0.2998,0.4556,0.3697,0.4631
6,0.7903,0.9351,0.9767,0.2967,0.4551,0.3682,0.4689
7,0.8050,0.9409,0.9805,0.3127,0.4741,0.3914,0.4882
8,0.7889,0.9287,0.9651,0.2947,0.4515,0.3637,0.4624


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('FeatureEnvy_lightgbm_FS2.latex',index=False)
dfi.export(df2, 'FeatureEnvy_lightgbm_FS2.png')

In [ ]:
plot_model(best_model_lightgbm, plot='feature',save=True)
os.rename('Feature Importance.png','FE_lightgbm_FS2.png')

In [ ]:
plot_model(best_model_xgboost, plot='feature',save=True)
os.rename('Feature Importance.png','FE_xgboost_FS2.png')

In [ ]:
plot_model(best_model_dt, plot='feature',save=True)
os.rename('Feature Importance.png','FE_dt_FS2.png')

In [ ]:
plot_model(best_model, plot='feature',save=True)
os.rename('Feature Importance.png','FE_RF_FS2.png')

In [ ]:
plot_model(best_model_et, plot='feature',save=True)
os.rename('Feature Importance.png','FE_et_FS2.png')

#### Ensemble the top-5 models

In [ ]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8148,0.9364,0.9845,0.3252,0.4889,0.4090,0.5030
1,0.8116,0.9239,0.9496,0.3174,0.4757,0.3940,0.4823
2,0.8134,0.9437,0.9922,0.3245,0.4890,0.4088,0.5049
3,0.8008,0.9306,0.9845,0.3094,0.4708,0.3868,0.4856
4,0.8082,0.9287,0.9729,0.3161,0.4772,0.3950,0.4890
5,0.8029,0.9400,0.9922,0.3117,0.4744,0.3914,0.4912
6,0.8165,0.9353,0.9922,0.3273,0.4923,0.4131,0.5082
7,0.8084,0.9239,0.9844,0.3170,0.4796,0.3979,0.4943
8,0.8227,0.9407,0.9767,0.3342,0.4980,0.4203,0.5098


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8128,0.9351,0.9801,0.3220,0.4848,0.4042,0.4981


In [ ]:
result = pull()

In [ ]:
result.to_csv('models_featureenvy_2.csv')

## Drop Code Smell - DataClass

In [ ]:
##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the DataClass smell. To train other models, remove the other code smells.

In [ ]:
#consider only the column 'dataclass_label'

df_dataclass = df.drop(columns=['longmethod_label','featureenvy_label','blob_label'])
df_dataclass.columns

Index(['dataclass_label', 'agreement_smell_type', 'type',
       'organic_weighofclass', 'organic_numberofaccessormethods',
       'organic_classeffectivelinesofcode', 'organic_lcom3',
       'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount',
       ...
       'smell_organic_dataclass', 'smell_organic_blob', 'smell_pmd_longmethod',
       'smell_pmd_featureenvy', 'smell_pmd_dataclass', 'smell_pmd_blob',
       'smell_designite_longmethod', 'smell_designite_featureenvy',
       'smell_designite_dataclass', 'smell_designite_blob'],
      dtype='object', length=142)

#### Check the correlation and remove the high-correlated features from the dataset

In [ ]:
corr = df_dataclass.corr(numeric_only = True)
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_dataclass.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_dataclass.columns[columns]
high_corr = set(df_dataclass.columns) - set(selected_columns)
df_dataclass = df_dataclass[selected_columns]
df_dataclass.columns

Index(['dataclass_label', 'agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethodsqt

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of DataClass.

In [ ]:
results = df_dataclass.dataclass_label.values
df_dataclass = df_dataclass.drop('dataclass_label', axis=1)
df_dataclass['dataclass_label'] = results
df_dataclass.columns

Index(['agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethodsqty',
       'ck_clas

#### Setup the model creation with pycaret

In [ ]:
s = setup(data=df_dataclass, target='dataclass_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Info] Number of positive: 23277, number of negative: 23277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020715 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4135
[LightGBM] [Info] Number of data points in the train set: 46554, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,1729
1,Target,dataclass_label
2,Target type,Binary
3,Original data shape,"(40951, 86)"
4,Transformed data shape,"(58840, 18)"
5,Transformed train set shape,"(46554, 18)"
6,Transformed test set shape,"(12286, 18)"
7,Numeric features,85
8,Rows with missing values,100.0%
9,Preprocess,True


In [ ]:
df_dataclass = get_config('X')
df_dataclass

,agreement_smell_type,organic_weighofclass,organic_numberofaccessormethods,organic_classeffectivelinesofcode,organic_lcom3,organic_tightclasscohesion,organic_isabstract,organic_publicfieldcount,organic_weightedmethodcount,organic_changingmethods,...,smell_organic_dataclass,smell_organic_blob,smell_pmd_longmethod,smell_pmd_featureenvy,smell_pmd_dataclass,smell_pmd_blob,smell_designite_longmethod,smell_designite_featureenvy,smell_designite_dataclass,smell_designite_blob
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [ ]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7900,0.8572,0.8823,0.4689,0.6123,0.4862,0.5319,2.7090
rf,Random Forest Classifier,0.7889,0.8554,0.8844,0.4676,0.6116,0.4850,0.5315,7.4580
et,Extra Trees Classifier,0.7889,0.8549,0.8788,0.4674,0.6102,0.4834,0.5288,6.2770
dt,Decision Tree Classifier,0.7837,0.8507,0.8796,0.4607,0.6046,0.4751,0.5222,2.8370
lightgbm,Light Gradient Boosting Machine,0.7851,0.8594,0.8692,0.4620,0.6033,0.4742,0.5188,5.1030
gbc,Gradient Boosting Classifier,0.7487,0.8127,0.7326,0.4066,0.5228,0.3707,0.4004,9.0690
ada,Ada Boost Classifier,0.7243,0.7606,0.6442,0.3670,0.4676,0.3000,0.3216,4.0380
knn,K Neighbors Classifier,0.7851,0.8150,0.4805,0.4352,0.4563,0.3229,0.3238,3.9900
ridge,Ridge Classifier,0.6791,0.6490,0.4816,0.2885,0.3607,0.1642,0.1738,2.3960
lda,Linear Discriminant Analysis,0.6781,0.6496,0.4824,0.2875,0.3603,0.1632,0.1730,2.6680


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('DataClass_FS2.latex',index=False)
df3.to_csv('DataClass_FS2.csv',index=False)
dfi.export(df2, 'DataClass_FS2.png')

#### Tune the models

In [ ]:
tuned_top5=[]
results=[]

In [ ]:
for model in top5_models:
    if model.__class__.__name__ == 'GaussianNB':
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=False  # Desativar early_stopping para Naive Bayes
        )
    else:
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=True
        )
    tuned_top5.append(tuned_model)
    best_model_results = pull()
    metrics = best_model_results.iloc[:, :-1].T.to_dict('list')
    params = tuned_model.get_params()

    result = {
        "Model": model.__class__.__name__,
        "Parameters": params
    }
    result.update(metrics)

    results.append(result)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7806,0.8649,0.9481,0.4595,0.6190,0.4898,0.5532
1,0.7824,0.8705,0.9796,0.4628,0.6286,0.5012,0.5717
2,0.7998,0.8788,0.9833,0.4840,0.6487,0.5304,0.5956
3,0.7862,0.8735,0.9759,0.4671,0.6318,0.5063,0.5746
4,0.7841,0.8664,0.9610,0.4642,0.6260,0.4989,0.5643
5,0.7865,0.8524,0.9740,0.4670,0.6313,0.5060,0.5737
6,0.7987,0.8788,0.9758,0.4821,0.6454,0.5263,0.5902
7,0.7690,0.8380,0.9759,0.4477,0.6138,0.4796,0.5538
8,0.8060,0.8860,0.9759,0.4920,0.6542,0.5389,0.6000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:optuna_integration.sklearn:Searching the best hyperparameters using 28665 samples...
INFO:optuna_integration.sklearn:Finished hyperparameter search!


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7824,0.8745,0.8924,0.4594,0.6066,0.4767,0.5269
1,0.7830,0.8860,0.9276,0.4617,0.6165,0.4880,0.5458
2,0.8022,0.8980,0.9369,0.4865,0.6405,0.5222,0.5756
3,0.7625,0.8655,0.9295,0.4379,0.5954,0.4564,0.5212
4,0.7803,0.8865,0.8998,0.4571,0.6062,0.4755,0.5279
5,0.8053,0.8890,0.9405,0.4903,0.6446,0.5281,0.5813
6,0.7990,0.8925,0.9368,0.4818,0.6364,0.5165,0.5710
7,0.7941,0.8882,0.9425,0.4761,0.6326,0.5102,0.5677
8,0.8130,0.9023,0.9276,0.5015,0.6510,0.5383,0.5858


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

INFO:optuna_integration.sklearn:Searching the best hyperparameters using 28665 samples...
INFO:optuna_integration.sklearn:Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7447,0.8689,0.9184,0.4184,0.5749,0.4269,0.4943
1,0.7405,0.8617,0.8998,0.4128,0.5659,0.4152,0.4794
2,0.7412,0.8658,0.9072,0.4141,0.5686,0.4185,0.4842
3,0.7412,0.8726,0.9369,0.4163,0.5765,0.4274,0.5004
4,0.7175,0.8625,0.8924,0.3901,0.5429,0.3809,0.4493
5,0.7627,0.8795,0.9498,0.4390,0.6005,0.4624,0.5322
6,0.7498,0.8717,0.8996,0.4220,0.5745,0.4284,0.4900
7,0.7306,0.8676,0.9555,0.4078,0.5716,0.4182,0.4990
8,0.7676,0.8776,0.8794,0.4409,0.5874,0.4494,0.5013


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:optuna_integration.sklearn:Searching the best hyperparameters using 28665 samples...
INFO:optuna_integration.sklearn:Finished hyperparameter search!


In [ ]:
df3 = pd.DataFrame(results)
df3.to_latex('DataClass_FS2_Top_5.latex',index=False)
dfi.export(df2, 'DataClass_FS2_Top_5.png')

#### Calculing top 5 most relevant features

##### Random Forest (RF)

In [ ]:
best_model = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7844,0.8521,0.8776,0.4615,0.6049,0.4756,0.5221
1,0.7907,0.8567,0.8850,0.4700,0.6139,0.4882,0.5342
2,0.7817,0.8433,0.8794,0.4580,0.6023,0.4717,0.5194
3,0.7911,0.8558,0.8609,0.4696,0.6077,0.4816,0.5227
4,0.7851,0.8453,0.8980,0.4632,0.6111,0.4828,0.5333
5,0.7962,0.8618,0.8848,0.4770,0.6198,0.4971,0.5414
6,0.7798,0.8445,0.8717,0.4549,0.5978,0.4661,0.5129
7,0.7777,0.8437,0.8683,0.4526,0.5950,0.4620,0.5086
8,0.7872,0.8491,0.8924,0.4656,0.6120,0.4846,0.5332


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('DataClass_RF_FS2.latex',index=False)
dfi.export(df2, 'DataClass_RF_FS2.png')

##### Extra Trees Classifier (ET)

In [ ]:
best_model_et= create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7841,0.8514,0.8757,0.4609,0.6040,0.4745,0.5207
1,0.7925,0.8571,0.8720,0.4719,0.6124,0.4873,0.5301
2,0.7844,0.8438,0.8850,0.4618,0.6069,0.4779,0.5259
3,0.7914,0.8571,0.8646,0.4702,0.6092,0.4833,0.5250
4,0.7817,0.8435,0.8794,0.4580,0.6023,0.4717,0.5194
5,0.7938,0.8580,0.8922,0.4738,0.6190,0.4952,0.5418
6,0.7795,0.8472,0.8680,0.4543,0.5964,0.4644,0.5105
7,0.7746,0.8405,0.8516,0.4478,0.5870,0.4518,0.4960
8,0.7858,0.8494,0.8831,0.4635,0.6079,0.4795,0.5267


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('DataClass_ET_FS2.latex',index=False)
dfi.export(df2, 'DataClass_ET_FS2.png')

##### Decision Tree (DT)

In [ ]:
best_model_dt= create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7792,0.8503,0.8850,0.4552,0.6011,0.4694,0.5190
1,0.7810,0.8504,0.8757,0.4569,0.6005,0.4694,0.5166
2,0.7747,0.8402,0.8757,0.4491,0.5937,0.4593,0.5084
3,0.7806,0.8491,0.8442,0.4550,0.5913,0.4592,0.5001
4,0.7782,0.8410,0.8813,0.4537,0.5990,0.4666,0.5157
5,0.7900,0.8575,0.8941,0.4688,0.6151,0.4893,0.5375
6,0.7736,0.8411,0.8606,0.4465,0.5879,0.4526,0.4990
7,0.7844,0.8379,0.8497,0.4603,0.5971,0.4671,0.5080
8,0.7833,0.8501,0.8924,0.4607,0.6077,0.4783,0.5282


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('DataClass_DT_FS2.latex',index=False)
dfi.export(df2, 'DataClass_DT_FS2.png')

##### Light Gradient Boosting Machine(lightgbm)

In [ ]:
best_model_lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7799,0.8504,0.8738,0.4555,0.5989,0.4672,0.5142
1,0.7890,0.8629,0.8738,0.4673,0.6089,0.4820,0.5263
2,0.7736,0.8455,0.8497,0.4464,0.5853,0.4496,0.4937
3,0.7883,0.8616,0.8534,0.4656,0.6025,0.4747,0.5152
4,0.7987,0.8796,0.8850,0.4808,0.6231,0.5017,0.5452
5,0.7924,0.8617,0.8643,0.4711,0.6098,0.4846,0.5260
6,0.7816,0.8504,0.8494,0.4561,0.5935,0.4621,0.5039
7,0.7851,0.8488,0.8534,0.4614,0.5990,0.4694,0.5109
8,0.7816,0.8504,0.8664,0.4574,0.5987,0.4677,0.5127


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('DataClass_lightgbm_FS2.latex',index=False)
dfi.export(df2, 'DataClass_lightgbm_FS2.png')

##### Extreme Gradient Boosting(xgboost)

In [ ]:
best_model_xgboost = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7817,0.8539,0.8794,0.4580,0.6023,0.4717,0.5194
1,0.7904,0.8595,0.8794,0.4693,0.6120,0.4860,0.5310
2,0.7824,0.8455,0.8794,0.4589,0.6031,0.4728,0.5203
3,0.7914,0.8584,0.8571,0.4700,0.6071,0.4811,0.5213
4,0.7858,0.8500,0.9054,0.4643,0.6138,0.4861,0.5380
5,0.7994,0.8582,0.8792,0.4812,0.6220,0.5008,0.5430
6,0.7812,0.8499,0.8643,0.4563,0.5973,0.4661,0.5109
7,0.7798,0.8434,0.8627,0.4550,0.5958,0.4637,0.5085
8,0.7802,0.8475,0.8831,0.4564,0.6018,0.4705,0.5194


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('DataClass_xgboost_FS2.latex',index=False)
dfi.export(df2, 'DataClass_xgboost_FS2.png')

In [ ]:
plot_model(best_model_lightgbm, plot='feature',save=True)
os.rename('Feature Importance.png','DT_lightgbm_FS2.png')

In [ ]:
plot_model(best_model_xgboost, plot='feature',save=True)
os.rename('Feature Importance.png','DT_xgboost_FS2.png')

In [ ]:
plot_model(best_model_dt, plot='feature',save=True)
os.rename('Feature Importance.png','DT_dt_FS2.png')

In [ ]:
plot_model(best_model, plot='feature',save=True)
os.rename('Feature Importance.png','DT_rf_FS2.png')

In [ ]:
plot_model(best_model_et, plot='feature',save=True)
os.rename('Feature Importance.png','DT_et_FS2.png')

#### Ensemble the top-5 models

In [ ]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

In [ ]:
pred = predict_model(blended_models)

In [ ]:
result = pull()

#### Plot the ensemble model

In [ ]:
result.to_csv('models_dataclas2.csv')

#### Finalize and save the ensemble model

In [ ]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_dataclass')

#### Interpret the ensemble model

In [ ]:
!pip install shap
interpret_model(tuned_top5[0], save=True)

## Drop Code Smell - Blob

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Blob. To train other models, remove the other code smells.

In [48]:
#consider only the column 'blob_label'

df_blob = df.drop(columns=['featureenvy_label','dataclass_label','longmethod_label'])
df_blob.columns

Index(['blob_label', 'agreement_smell_type', 'type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount',
       ...
       'smell_organic_dataclass', 'smell_organic_blob', 'smell_pmd_longmethod',
       'smell_pmd_featureenvy', 'smell_pmd_dataclass', 'smell_pmd_blob',
       'smell_designite_longmethod', 'smell_designite_featureenvy',
       'smell_designite_dataclass', 'smell_designite_blob'],
      dtype='object', length=142)

#### Check the correlation and remove the high-correlated features from the dataset

In [49]:
corr = df_blob.corr(numeric_only = True)
threshold = 0.9
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_blob.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_blob.columns[columns]
high_corr = set(df_blob.columns) - set(selected_columns)
df_blob = df_blob[selected_columns]
df_blob.columns

Index(['blob_label', 'agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethodsqty',
 

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Blob.

In [50]:
results = df_blob.blob_label.values
df_blob = df_blob.drop('blob_label', axis=1)
df_blob['blob_label'] = results
df_blob.columns

Index(['agreement_smell_type', 'organic_weighofclass',
       'organic_numberofaccessormethods', 'organic_classeffectivelinesofcode',
       'organic_lcom3', 'organic_tightclasscohesion', 'organic_isabstract',
       'organic_publicfieldcount', 'organic_weightedmethodcount',
       'organic_changingmethods', 'organic_numberoftrystatements',
       'organic_maxnesting', 'organic_numberoffinallystatements',
       'organic_numberofthrowstatements',
       'organic_numberoftrystatementswithnocatchandfinally',
       'organic_cyclomaticcomplexity', 'organic_couplingintensity',
       'organic_methodeffectivelinesofcode', 'organic_couplingdispersion',
       'organic_exceptionalloc', 'organic_numberofdummyexceptionhandlers',
       'organic_changingclasses', 'organic_maxcallchain',
       'organic_thrownexceptiontypescount', 'ck_class_cbo', 'ck_class_fanin',
       'ck_class_lcom', 'ck_class_totalmethodsqty',
       'ck_class_privatemethodsqty', 'ck_class_defaultmethodsqty',
       'ck_clas

#### Setup the model creation with pycaret

In [51]:
s = setup(data=df_blob, target='blob_label', feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Info] Number of positive: 23652, number of negative: 23652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4911
[LightGBM] [Info] Number of data points in the train set: 47304, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,7813
1,Target,blob_label
2,Target type,Binary
3,Original data shape,"(40951, 86)"
4,Transformed data shape,"(59590, 18)"
5,Transformed train set shape,"(47304, 18)"
6,Transformed test set shape,"(12286, 18)"
7,Numeric features,85
8,Rows with missing values,100.0%
9,Preprocess,True


In [52]:
df_blob = get_config('X')
df_blob

,agreement_smell_type,organic_weighofclass,organic_numberofaccessormethods,organic_classeffectivelinesofcode,organic_lcom3,organic_tightclasscohesion,organic_isabstract,organic_publicfieldcount,organic_weightedmethodcount,organic_changingmethods,...,smell_organic_dataclass,smell_organic_blob,smell_pmd_longmethod,smell_pmd_featureenvy,smell_pmd_dataclass,smell_pmd_blob,smell_designite_longmethod,smell_designite_featureenvy,smell_designite_dataclass,smell_designite_blob
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40946,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40947,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40948,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
40949,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [53]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8355,0.9126,0.8713,0.5185,0.6498,0.5513,0.5833,6.5080
lightgbm,Light Gradient Boosting Machine,0.8357,0.9176,0.8681,0.5185,0.6491,0.5507,0.5819,5.2330
rf,Random Forest Classifier,0.8325,0.9154,0.8793,0.5145,0.6483,0.5484,0.5830,8.1870
xgboost,Extreme Gradient Boosting,0.8307,0.9121,0.8761,0.5104,0.6446,0.5437,0.5781,2.8470
dt,Decision Tree Classifier,0.8261,0.9076,0.8759,0.5042,0.6390,0.5357,0.5718,2.5270
gbc,Gradient Boosting Classifier,0.8051,0.8768,0.7556,0.4649,0.5756,0.4582,0.4811,9.4320
knn,K Neighbors Classifier,0.8434,0.8702,0.5099,0.5573,0.5324,0.4386,0.4393,4.0980
ada,Ada Boost Classifier,0.7741,0.8434,0.7112,0.4170,0.5248,0.3902,0.4147,4.3660
ridge,Ridge Classifier,0.8247,0.7637,0.4125,0.4999,0.4516,0.3485,0.3510,2.4910
lda,Linear Discriminant Analysis,0.8189,0.7622,0.4137,0.4821,0.4445,0.3372,0.3391,2.6250


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Blob_RF_FS2.latex',index=False)
df3.to_csv('Blob_RF_FS2.csv',index=False)
dfi.export(df2,'Blob_RF_FS2.png')

#### Tune the models

In [ ]:
tuned_top5=[]
results=[]

In [ ]:
for model in top5_models:
    if model.__class__.__name__ == 'GaussianNB':
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=False  # Desativar early_stopping para Naive Bayes
        )
    else:
        tuned_model = tune_model(
            model,
            n_iter=30,
            optimize="f1",
            search_library='optuna',
            choose_better=True,
            early_stopping=True
        )
    tuned_top5.append(tuned_model)
    best_model_results = pull()
    metrics = best_model_results.iloc[:, :-1].T.to_dict('list')
    params = tuned_model.get_params()

    result = {
        "Model": model.__class__.__name__,
        "Parameters": params
    }
    result.update(metrics)

    results.append(result)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8287,0.9026,0.9721,0.5052,0.6648,0.5648,0.6193
1,0.8249,0.9092,0.9780,0.4995,0.6613,0.5593,0.6167
2,0.8389,0.9185,0.9741,0.5213,0.6792,0.5844,0.6353
3,0.8123,0.8925,0.9681,0.4821,0.6437,0.5350,0.5949
4,0.8225,0.9015,0.9900,0.4965,0.6613,0.5583,0.6194
5,0.8332,0.9087,0.9741,0.5121,0.6713,0.5735,0.6268
6,0.8283,0.9119,0.9641,0.5047,0.6626,0.5621,0.6150
7,0.8158,0.8986,0.9581,0.4863,0.6452,0.5380,0.5944
8,0.8280,0.9106,0.9741,0.5041,0.6644,0.5639,0.6192


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:optuna_integration.sklearn:Searching the best hyperparameters using 28665 samples...
INFO:optuna_integration.sklearn:Finished hyperparameter search!


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8256,0.9082,0.9621,0.5005,0.6585,0.5565,0.6101
1,0.8340,0.9307,0.9621,0.5133,0.6694,0.5719,0.6222
2,0.8354,0.9280,0.9502,0.5162,0.6690,0.5719,0.6190
3,0.8221,0.9258,0.9661,0.4959,0.6554,0.5517,0.6074
4,0.8333,0.9255,0.9781,0.5125,0.6726,0.5749,0.6290
5,0.8346,0.9205,0.9661,0.5143,0.6713,0.5741,0.6250
6,0.8371,0.9254,0.9561,0.5184,0.6723,0.5762,0.6240
7,0.8203,0.9150,0.9661,0.4929,0.6527,0.5481,0.6046
8,0.8283,0.9213,0.9621,0.5047,0.6621,0.5615,0.6140


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

INFO:optuna_integration.sklearn:Searching the best hyperparameters using 28665 samples...
INFO:optuna_integration.sklearn:Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7942,0.9111,0.9481,0.4572,0.6169,0.4987,0.5607
1,0.8047,0.9144,0.9561,0.4710,0.6311,0.5183,0.5784
2,0.7855,0.9051,0.9422,0.4466,0.6060,0.4833,0.5468
3,0.7973,0.9133,0.9502,0.4618,0.6215,0.5048,0.5661
4,0.7970,0.9061,0.9661,0.4619,0.6250,0.5086,0.5737
5,0.8070,0.9117,0.9521,0.4742,0.6330,0.5213,0.5796
6,0.8126,0.9122,0.9421,0.4816,0.6374,0.5283,0.5823
7,0.7990,0.9104,0.9521,0.4636,0.6235,0.5078,0.5690
8,0.8116,0.9138,0.9481,0.4803,0.6376,0.5281,0.5838


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:optuna_integration.sklearn:Searching the best hyperparameters using 28665 samples...
INFO:optuna_integration.sklearn:Finished hyperparameter search!


In [ ]:
df3 = pd.DataFrame(results)
df3.to_latex('Blob_FS2.latex',index=False)
dfi.export(df2,'Blob_FS2.png')

#### Calculing top 5 most relevant features

##### Random Forest (RF)

In [ ]:
best_model = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8396,0.9063,0.8483,0.5253,0.6489,0.5522,0.5788
1,0.8465,0.9264,0.8882,0.5368,0.6692,0.5770,0.6080
2,0.8319,0.9034,0.8466,0.5120,0.6381,0.5371,0.5659
3,0.8232,0.9000,0.8367,0.4970,0.6236,0.5176,0.5476
4,0.7928,0.9113,0.9402,0.4556,0.6138,0.4946,0.5552
5,0.7948,0.9073,0.9182,0.4568,0.6101,0.4913,0.5465
6,0.8283,0.8984,0.8583,0.5053,0.6361,0.5334,0.5655
7,0.8364,0.9054,0.8543,0.5194,0.6460,0.5477,0.5764
8,0.8301,0.9054,0.8583,0.5083,0.6385,0.5367,0.5682


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Blob_RF_FS2.latex',index=False)
dfi.export(df2, 'Blob_RF_FS2.png')

##### Extra Trees Classifier (ET)

In [ ]:
best_model_et= create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8483,0.9137,0.8403,0.5425,0.6594,0.5675,0.5900
1,0.8518,0.9274,0.8862,0.5468,0.6763,0.5871,0.6159
2,0.8336,0.9044,0.8406,0.5153,0.6389,0.5388,0.5660
3,0.8249,0.8967,0.8347,0.5000,0.6254,0.5203,0.5494
4,0.8361,0.9111,0.8984,0.5184,0.6574,0.5597,0.5962
5,0.8451,0.9087,0.8603,0.5354,0.6600,0.5666,0.5934
6,0.8304,0.8980,0.8483,0.5090,0.6362,0.5345,0.5642
7,0.8371,0.9058,0.8343,0.5212,0.6416,0.5433,0.5685
8,0.8290,0.9032,0.8503,0.5065,0.6349,0.5324,0.5629


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Blob_ET_FS2.latex',index=False)
dfi.export(df2, 'Blob_ET_FS2.png')

##### Decision Tree (DT)

In [ ]:
best_model_dt= create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8486,0.9121,0.8383,0.5433,0.6593,0.5677,0.5897
1,0.8469,0.9241,0.8862,0.5375,0.6692,0.5772,0.6077
2,0.8294,0.8998,0.8307,0.5079,0.6304,0.5278,0.5547
3,0.8246,0.8941,0.8287,0.4994,0.6232,0.5179,0.5461
4,0.7953,0.9193,0.9402,0.4587,0.6166,0.4986,0.5583
5,0.8412,0.9064,0.8523,0.5285,0.6524,0.5568,0.5835
6,0.8276,0.8957,0.8603,0.5041,0.6357,0.5327,0.5653
7,0.8329,0.9052,0.8483,0.5133,0.6396,0.5392,0.5680
8,0.8280,0.9010,0.8543,0.5047,0.6345,0.5316,0.5631


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Blob_DT_FS2.latex',index=False)
dfi.export(df2, 'Blob_DT_FS2.png')

##### Light Gradient Boosting Machine(lightgbm)

In [ ]:
best_model_lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8305,0.9070,0.8443,0.5090,0.6351,0.5334,0.5624
1,0.8486,0.9281,0.8842,0.5409,0.6712,0.5802,0.6097
2,0.8333,0.9028,0.8347,0.5147,0.6368,0.5363,0.5627
3,0.8193,0.8993,0.8327,0.4906,0.6174,0.5093,0.5398
4,0.8396,0.9219,0.8785,0.5250,0.6572,0.5610,0.5927
5,0.7906,0.9104,0.9182,0.4514,0.6053,0.4844,0.5410
6,0.8234,0.9015,0.8503,0.4971,0.6274,0.5219,0.5542
7,0.8360,0.9052,0.8563,0.5187,0.6461,0.5476,0.5767
8,0.8322,0.9092,0.8782,0.5116,0.6466,0.5464,0.5807


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Blob_lightgbm_FS2.latex',index=False)
dfi.export(df2, 'Blob_lightgbm_FS2.png')

##### Extreme Gradient Boosting(xgboost)

In [ ]:
best_model_xgboost = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8389,0.9084,0.8583,0.5238,0.6505,0.5537,0.5822
1,0.7946,0.9143,0.9421,0.4574,0.6158,0.4976,0.5581
2,0.8340,0.9043,0.8606,0.5155,0.6448,0.5452,0.5756
3,0.8287,0.8996,0.8466,0.5066,0.6339,0.5311,0.5610
4,0.8385,0.9130,0.8904,0.5228,0.6588,0.5622,0.5964
5,0.7868,0.9091,0.9122,0.4463,0.5993,0.4764,0.5329
6,0.8322,0.9020,0.8603,0.5119,0.6418,0.5413,0.5724
7,0.8318,0.9039,0.8563,0.5113,0.6403,0.5395,0.5700
8,0.8381,0.9173,0.8762,0.5220,0.6542,0.5572,0.5891


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Blob_xgboost_FS2.latex',index=False)
dfi.export(df2, 'Blob_xgboost_FS2.png')

In [ ]:
plot_model(best_model_xgboost, plot='feature',save=True)
os.rename('Feature Importance.png','Blob_xgboost_FS2.png')


In [ ]:
plot_model(best_model_lightgbm, plot='feature',save=True)
os.rename('Feature Importance.png','Blob_lightgbm_FS2.png')

In [ ]:
plot_model(best_model_dt, plot='feature',save=True)
os.rename('Feature Importance.png','Blob_dt_FS2.png')

In [ ]:
plot_model(best_model, plot='feature',save=True)
os.rename('Feature Importance.png','Blob_rf_FS2.png')

In [ ]:
plot_model(best_model_et, plot='feature',save=True)
os.rename('Feature Importance.png','Blob_et_FS2.png')

#### Ensemble the top-5 models

In [54]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8427,0.9175,0.8643,0.5306,0.6576,0.5629,0.5911
1,0.8298,0.8973,0.8523,0.5077,0.6364,0.5344,0.5649
2,0.8294,0.9104,0.8546,0.5077,0.6370,0.5348,0.5657
3,0.8036,0.9220,0.9522,0.4700,0.6294,0.5158,0.5753
4,0.7939,0.9079,0.8984,0.4551,0.6042,0.4843,0.5355
5,0.8391,0.9155,0.8723,0.5240,0.6547,0.5582,0.5890
6,0.8538,0.9279,0.8802,0.5512,0.6779,0.5897,0.6168
7,0.8426,0.9214,0.8643,0.5306,0.6576,0.5629,0.5911
8,0.7994,0.9243,0.9301,0.4632,0.6184,0.5023,0.5585


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [55]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8433,0.9154,0.8706,0.5317,0.6602,0.5660,0.5950


In [56]:
result = pull()

#### Plot the ensemble model

In [57]:
result.to_csv('models_blob2.csv')

#### Finalize and save the ensemble model

In [58]:
final_best = finalize_model(blended_models)
save_model(final_best, 'model_blob2')

IndexError: Length of values (67580) does not match length of index (40951). This usually happens when transformations that drop rows aren't applied on all the columns.